A number of students and a number of projects. 

Any student can be assigned any project, incurring some cost that may vary depending on the student-project assignment. 

Number of projects > number of students

A project with N students permitted is represented as N seperate projects, each with equal ranking by any student who chooses that project. 

In [54]:
import numpy as np
import random
from scipy.optimize import linear_sum_assignment
import csv

End of file name for data to be fed to model

In [62]:
# fname = 'ideal' # first choice of each student uniquely ranked 1

fname = '0'       # randomly generated seelction form

fname = '2020_21' # data from previous years dissertation unit 

# Student Project Assignment

In [63]:
def build_cost_matrix(n_students, n_projects, n_choices):
    """
    Creates a student project selection form
    """
    students = list(range(n_students))
    projects = list(range(n_projects))

    cost_matrix = []
    
    for s in students:
        selections = [10 for p in projects] # assigns a suitably high number to all unselected projects
        
        # selects n ranked choices per student for projects selected randomly from a weighted probability distribution
        for n in range(1, n_choices+1):
            
            element = random.choices(np.arange(n_projects), weights=np.linspace(0, 1, n_projects))
            
            selections[element[0]] = n
            
        # repeat the last two entries to represent two projects that can each accept 2 students 
        selections = selections + selections[-2:] 
            
        cost_matrix.append(selections)
    
    return np.array(cost_matrix)

Build student preference matrix

In [64]:
# # build student project selection form using random values
# n_students = 10
# n_projects = 15
# n_choices = 5
# cost_matrix = build_cost_matrix(n_students, n_projects)                      

# build student project selection form by importing
with open(f'data/student_preference_matrices/student_preference_matrix_{fname}.csv', 'r') as f:   
        
        cost_matrix = csv.reader(f)
        
        cost_matrix = [[int(i) for i in j] for j in cost_matrix]            # convert from string
        
        cost_matrix = [[10 if i==0 else i for i in j] for j in cost_matrix] # replace zeros with 10
        
        cost_matrix = [j+j[-2:] for j in cost_matrix]                       # repeat last two entries to simulate 2 projects with student capacity 2
        
#         cost_matrix = np.array(cost_matrix)
        

print('\ncost matrix\n', np.matrix(cost_matrix))


cost matrix
 [[10 10 10 ... 10 10 10]
 [10 10 10 ... 10 10 10]
 [10 10 10 ... 10 10 10]
 ...
 [10 10 10 ... 10 10 10]
 [10 10 10 ... 10  3 10]
 [10 10 10 ... 10 10 10]]


In [65]:
n_students = len(cost_matrix)
print(n_students)

n_projects = len(cost_matrix[0])
print(n_projects)

74
302


Modify form to include projects with student capacity > 1 by creating repeat instances of the project in the form, with equal student ranking 

In [66]:
with open(f'data/student_preference_matrices/project_capacity_{fname}.csv', 'r') as f:
    capacity = list(csv.reader(f))
    capacity = [int(i) for i in capacity[0]]
    
print(len(capacity))

    
# give projects an ID so that multiple columns with the same ID can be re-grouped later 
project_ID = list(range(n_projects)) 

# temporarily include project ID and capacity vectors in cost matrix
cost_matrix.insert(0, project_ID)
cost_matrix.insert(1, capacity)


# if project capacity n > 1
for i in range(n_projects):              
    if capacity[i] > 1:
        
        # create n-1 copies of the column for that project
        # there are now n columns representing that project (with same ID number and student rank)
        for j in range(capacity[i]-1):   
            for row in cost_matrix:
                row.append(row[i])
                

# remove adn store project ID and capacity vectors 
project_ID = cost_matrix.pop(0)
capacity = cost_matrix.pop(0)
                
print(np.matrix(cost_matrix))

299


IndexError: list index out of range

Convert cost matrix to Numpy array

In [49]:
cost_matrix = np.array(cost_matrix)

Find optimal solution - minimize linear sum 

In [53]:
studentID, projectID = linear_sum_assignment(cost_matrix) 

print(studentID, projectID)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73] [275  72 223 274 228  52 128 146  16 276 188 255  64  37 107 256 285  51
 266 244 166  45 216 209 284 127 231  73  18  28   4  35 217  71 206 101
 245 164 158 260 133 251   8  96 241 297 281 222  48 272 220 299 258 253
  49 262 249 288 286 230  83 139 246  65 290  79 170 144 137 294 257 296
  66  61]


Check every student has been asisgned a project

In [51]:
assign_matrix = np.zeros(cost_matrix.shape)    # create an array to display assignments

for s, p in zip(studentID, projectID):         # mark each assignment with a 1
    assign_matrix[s, p] = 1
    
# check every student has a project
print('\ntotal_projects_assigned=',np.sum(assign_matrix), '\nequal to number of students=', np.sum(assign_matrix)==n_students)


total_projects_assigned= 74.0 
equal to number of students= False


Display output

In [52]:
assign_matrix *= cost_matrix                                     # matrix showing cost of each assignment 

# students 
print('student', list(range(n_students)))

# project ID of each project in best solution
allocations = [project_ID[j] for j in np.where(assign_matrix)[0]]
print('project allocation', allocations)

# student ranking of each project in best solution
ranks = assign_matrix[np.nonzero(assign_matrix)]
print('rank of allocated project', ranks)
print('\nsum of project rankings (should be as low as possible)=', sum(ranks))

print('\nassign matrix\n',np.matrix(np.absolute(assign_matrix)))
# print('\nstudents', studentID)
# print('project ID (this model)', projectID)
# print('project rank', ranks)
# print('\n\nsum of all assignments (should be as low as possible)=', ranks.sum())

student [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


IndexError: list index out of range